In [37]:
###Install the requirements
!pip3 install geocoder
!pip3 install fastparquet
!pip3 install -U google-api-core
!pip3 install --upgrade pip
!pip3 install --upgrade google-cloud-language
!pip3 install --upgrade google-cloud-bigquery
!pip3 install -t lib/ --upgrade google-api-python-client
!pip3 install -t lib/ --upgrade google-cloud

    100% |████████████████████████████████| 153kB 7.6MB/s ta 0:00:01
    100% |████████████████████████████████| 3.3MB 8.2MB/s eta 0:00:01    23% |███████▋                        | 788kB 29.4MB/s eta 0:00:01
    100% |████████████████████████████████| 16.8MB 1.8MB/s eta 0:00:01
  Running setup.py bdist_wheel for fastparquet ... done
  Stored in directory: /content/.cache/pip/wheels/f3/27/fb/839c776ec8689ff9ee52ad3e91d7d4d848ac6d7545a127d5b0
Successfully built fastparquet
    100% |████████████████████████████████| 71kB 3.2MB/s ta 0:00:011
google-cloud-bigquery 1.8.1 has requirement google-cloud-core<0.30dev,>=0.29.0, but you'll have google-cloud-core 0.28.1 which is incompatible.
google-cloud-monitoring 0.28.0 has requirement google-api-core<0.2.0dev,>=0.1.1, but you'll have google-api-core 1.10.0 which is incompatible.
  Found existing installation: google-api-core 0.1.4
    Uninstalling google-api-core-0.1.4:
      Successfully uninstalled google-api-core-0.1.4
    100% |████████████

     |████████████████████████████████| 133kB 3.5MB/s eta 0:00:01
ERROR: google-cloud-monitoring 0.28.0 has requirement google-api-core<0.2.0dev,>=0.1.1, but you'll have google-api-core 1.10.0 which is incompatible.
ERROR: google-cloud-monitoring 0.28.0 has requirement google-cloud-core<0.29dev,>=0.28.0, but you'll have google-cloud-core 0.29.1 which is incompatible.
  Found existing installation: google-cloud-core 0.28.1
    Uninstalling google-cloud-core-0.28.1:
      Successfully uninstalled google-cloud-core-0.28.1
  Found existing installation: google-cloud-bigquery 1.8.1
    Uninstalling google-cloud-bigquery-1.8.1:
      Successfully uninstalled google-cloud-bigquery-1.8.1
     |████████████████████████████████| 61kB 2.8MB/s eta 0:00:011
     |████████████████████████████████| 102kB 9.1MB/s ta 0:00:011


     |████████████████████████████████| 81kB 5.0MB/s eta 0:00:011     |██████████████████████▎         | 51kB 5.5MB/s eta 0:00:01
     |████████████████████████████████| 81kB 17.4MB/s eta 0:00:01
     |████████████████████████████████| 81kB 12.3MB/s eta 0:00:01
ERROR: tensorboard 1.8.0 has requirement bleach==1.5.0, but you'll have bleach 2.1.2 which is incompatible.
ERROR: tensorboard 1.8.0 has requirement html5lib==0.9999999, but you'll have html5lib 1.0.1 which is incompatible.
ERROR: google-cloud-monitoring 0.28.0 has requirement google-api-core<0.2.0dev,>=0.1.1, but you'll have google-api-core 1.10.0 which is incompatible.
ERROR: google-cloud-monitoring 0.28.0 has requirement google-cloud-core<0.29dev,>=0.28.0, but you'll have google-cloud-core 0.29.1 which is incompatible.
ERROR: datalab 1.1.3 has requirement six==1.10.0, but you'll have six 1.12.0 which is incompatible.


In [1]:
###Import the requirements
import geocoder
from google.cloud import language_v1
from google.cloud.language_v1 import enums
from google.cloud.language import types
import six
from google.cloud import bigquery
import pandas as pd
client = bigquery.Client(location='US')
client2 = language_v1.LanguageServiceClient()

In [19]:
###Define the functions -  in this case we're using the geoparser as well as the sentiment analysis
def coordenadas_geonames(text):
  """Uses GeoNames parser to return the coordinates from a given text"""
  if text==None:
    return None 
  else:
    return geocoder.geonames(text, key='guticamilo').latlng

def classify_score(num, threshold=0.3):
  """Classifies sentiment according to a threshold set"""
  if (num>threshold):
    return "positive"
  elif (num<-threshold):
    return "negative"
  else:
    return "neutral"
  
def generate_sentiment_score(content, lang="es", client_type=client2):
  """Inputs: text (content, language to return the sentiment analysis (lang), and the client generated for the sentiment analysis (client_type)
      Outputs:the sentiment score (from -1 to 1, 1 being absolute positive and -1 being absolutely negative)"""
  if isinstance(content, six.binary_type):
        content = content.decode('utf-8')
  type_ = enums.Document.Type.PLAIN_TEXT
  document = {'type': type_, 'content': content, "language": lang}
  response = client_type.analyze_sentiment(document)
  sentiment = response.document_sentiment
  return sentiment.score

In [10]:
print(coordenadas_geonames('peru'))

['-10', '-75.25']


In [9]:
###Querying the Dataset
sql = """
select 
created_at,
text,
source,
user.location,
user.verified,
user.followers_count,
user.lang
time_zone,
place,
place.full_name,
entities.hashtags,
lang
from `servinf-geomark-social-prod.Twittero.tweets`
where lang="es"

"""
df = client.query(sql).to_dataframe()
df.head()

,created_at,text,source,location,verified,followers_count,time_zone,place,full_name,hashtags,lang
0,2019-05-14 14:56:42+00:00,RT @acostasamu: #EstimadorNegocios variación i...,"<a href=""http://twitter.com/download/android"" ...",Valenzuela- Paraguay.,False,281,es,None,None,"[{'indices': [16, 34], 'text': 'EstimadorNegoc...",es
1,2019-05-14 15:50:36+00:00,RT @acostasamu: #EstimadorNegocios variación i...,"<a href=""http://twitter.com/download/iphone"" r...","ÜT: -25.294094,-57.582651",False,689,es,None,None,"[{'indices': [16, 34], 'text': 'EstimadorNegoc...",es
2,2019-05-14 15:12:23+00:00,Accede a la aldeaswinger ahora! Costo 30 soles...,"<a href=""http://twitter.com/#!/download/ipad"" ...",Peru,False,1025,es,None,None,[],es
3,2019-05-14 15:38:20+00:00,RT @im_carlitox: Viste la recesión en la que e...,"<a href=""http://twitter.com/download/android"" ...",None,False,44,es,None,None,[],es
4,2019-05-14 15:09:54+00:00,RT @acostasamu: #EstimadorNegocios variación i...,"<a href=""http://twitter.com/download/android"" ...",Paraguay,False,604,es,None,None,"[{'indices': [16, 34], 'text': 'EstimadorNegoc...",es


In [11]:
###Creating additional functions using the created functions
df['COORD'] = df['location'].apply(coordenadas_geonames)


Error query term is missing. from JSON {'status': {'value': 14, 'message': 'query term is missing.'}}
Error query term is missing. from JSON {'status': {'value': 14, 'message': 'query term is missing.'}}
Error query term is missing. from JSON {'status': {'value': 14, 'message': 'query term is missing.'}}
Error query term is missing. from JSON {'status': {'value': 14, 'message': 'query term is missing.'}}
Error query term is missing. from JSON {'status': {'value': 14, 'message': 'query term is missing.'}}
Error query term is missing. from JSON {'status': {'value': 14, 'message': 'query term is missing.'}}
Error query term is missing. from JSON {'status': {'value': 14, 'message': 'query term is missing.'}}
Error query term is missing. from JSON {'status': {'value': 14, 'message': 'query term is missing.'}}
Error query term is missing. from JSON {'status': {'value': 14, 'message': 'query term is missing.'}}
Error query term is missing. from JSON {'status': {'value': 14, 'message': 'query 

Error query term is missing. from JSON {'status': {'value': 14, 'message': 'query term is missing.'}}
Error query term is missing. from JSON {'status': {'value': 14, 'message': 'query term is missing.'}}
Error query term is missing. from JSON {'status': {'value': 14, 'message': 'query term is missing.'}}
Error query term is missing. from JSON {'status': {'value': 14, 'message': 'query term is missing.'}}
Error query term is missing. from JSON {'status': {'value': 14, 'message': 'query term is missing.'}}
Error query term is missing. from JSON {'status': {'value': 14, 'message': 'query term is missing.'}}
Error query term is missing. from JSON {'status': {'value': 14, 'message': 'query term is missing.'}}
Error query term is missing. from JSON {'status': {'value': 14, 'message': 'query term is missing.'}}
Error query term is missing. from JSON {'status': {'value': 14, 'message': 'query term is missing.'}}
Error query term is missing. from JSON {'status': {'value': 14, 'message': 'query 

In [12]:
df['COORD'].head()

0    [-25.58852, -56.87065]
1                      None
2             [-10, -75.25]
3                      None
4          [-23.33333, -58]
Name: COORD, dtype: object

In [13]:
df['Score'] = df['text'].apply(generate_sentiment_score)

In [14]:
df['Score'].head()

0    0.2
1    0.2
2    0.0
3    0.0
4    0.2
Name: Score, dtype: float64

In [21]:
df['Sentiment'] = df['Score'].apply(classify_score)

In [22]:
df.head()

,created_at,text,source,location,verified,followers_count,time_zone,place,full_name,hashtags,lang,COORD,Score,Sentiment
0,2019-05-14 14:56:42+00:00,RT @acostasamu: #EstimadorNegocios variación i...,"<a href=""http://twitter.com/download/android"" ...",Valenzuela- Paraguay.,False,281,es,None,None,"[{'indices': [16, 34], 'text': 'EstimadorNegoc...",es,"[-25.58852, -56.87065]",0.2,neutral
1,2019-05-14 15:50:36+00:00,RT @acostasamu: #EstimadorNegocios variación i...,"<a href=""http://twitter.com/download/iphone"" r...","ÜT: -25.294094,-57.582651",False,689,es,None,None,"[{'indices': [16, 34], 'text': 'EstimadorNegoc...",es,None,0.2,neutral
2,2019-05-14 15:12:23+00:00,Accede a la aldeaswinger ahora! Costo 30 soles...,"<a href=""http://twitter.com/#!/download/ipad"" ...",Peru,False,1025,es,None,None,[],es,"[-10, -75.25]",0.0,neutral
3,2019-05-14 15:38:20+00:00,RT @im_carlitox: Viste la recesión en la que e...,"<a href=""http://twitter.com/download/android"" ...",None,False,44,es,None,None,[],es,None,0.0,neutral
4,2019-05-14 15:09:54+00:00,RT @acostasamu: #EstimadorNegocios variación i...,"<a href=""http://twitter.com/download/android"" ...",Paraguay,False,604,es,None,None,"[{'indices': [16, 34], 'text': 'EstimadorNegoc...",es,"[-23.33333, -58]",0.2,neutral


In [24]:
###Adding an output table to bq
dataset_id="Twittero"
dataset = client.dataset(dataset_id)
table_ref = dataset.table('outputs1')
load_job = client.load_table_from_dataframe(df, table_ref)
load_job.result()  # Waits for table load to complete.
